In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
plt.style.use('seaborn')
sns.set(font_scale=2)

In [2]:
sensor_df = pd.read_csv('./data/sensor.raw', names=('date', 'indicator'), parse_dates = ['date'])
cont_df = pd.read_csv('./data/cont.raw', names=('indicator', 'counter_', 'date'), parse_dates = ['date'])

In [3]:
sensor_df.loc[175, 'indicator'] = 89
sensor_df.indicator = pd.to_numeric(sensor_df.indicator)
MIN_DATE = min(sensor_df.date.min(), cont_df.date.min()).replace(hour=0, minute=0, second=0, microsecond=0)

In [4]:
counter_df = pd.DataFrame()
counter_df[['indicator', 'date']] = cont_df[['counter_', 'date']].copy()
#cont_df = cont_df.drop('counter_', axis=1)

sensor_df['Датчик'] = 'sensor'
cont_df['Датчик'] = 'cont'
counter_df['Датчик'] = 'счетчик остановок'

In [6]:
cont_df

,indicator,counter_,date,Датчик
0,0.7500,1,2021-03-15 01:46:21,cont
1,1.5000,2,2021-03-15 01:46:57,cont
2,2.2500,3,2021-03-15 01:54:25,cont
3,3.0000,4,2021-03-15 01:54:25,cont
4,3.7500,5,2021-03-15 01:54:25,cont
...,...,...,...,...
57,43.5000,58,2021-03-15 04:21:30,cont
58,43.6875,59,2021-03-15 04:21:30,cont
59,44.6250,60,2021-03-15 04:56:03,cont
60,45.4500,61,2021-03-15 07:03:29,cont


In [5]:
df = pd.concat([sensor_df, cont_df['indicator', 'date', 'Датчик'], counter_df], axis=0)
df = df.reset_index()
df['hours_'] = (df.date - MIN_DATE).dt.seconds / 3600

KeyError: ('indicator', 'date', 'Датчик')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.lineplot(data=df, x='hours_', y='indicator', hue='Датчик')
ax.set_title('Линейный график показаний датчиков')
ax.set_ylabel('Показания датчиков')
ax.set_xlabel('Время суток(часы)')
plt.savefig('./data/linear.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(x=df.hours_, y=df.indicator, hue=df['Датчик'])
ax.set_title('Точечный график показаний датчиков')
ax.set_ylabel('Показания датчиков')
ax.set_xlabel('Время суток(часы)')
plt.savefig('./data/scatter.png')
plt.show()

In [ ]:
# df.to_excel('output.xlsx')

In [ ]:
#sensor33_df['hours_'] = (sensor33_df.date - MIN_DATE).dt.seconds / 3600
#cont33_df['hours_'] = (cont33_df.date - MIN_DATE).dt.seconds / 3600

#sensor33_df.to_excel('sensor33.xlsx')
#cont33_df.to_excel('cont33.xlsx')